In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [56]:
class hybrid(object):
    
    def __init__ (self,user_id,ratings):
        
        self.user_id = user_id
        self.md = pd.read_csv('dataset/processed/FinalData.csv')
        self.ratings = ratings
        self.res=None
        print(ratings)
        print(ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']])
        print('STARTING POPULARITY')
        self.popularity_rating = self.popularity(self.md)
        print('STARTING CONTENT')
        self.content_rating = self.content_based(self.md,self.ratings,self.user_id)
        print('STARTING COLABORATIVE')
        self.collaborative_rating = self.collaborative(self.ratings, self.user_id)
        print('STARTING HYBRID')
        self.final_hybrid(self.md, self.popularity_rating , self.collaborative_rating, self.content_rating, self.user_id)
        
        
    #Popularity#

    def popularity(self,md):


        fd = pd.read_csv('dataset/processed/AverageRatings.csv')
        fd1 = pd.read_csv('dataset/processed/RatingsCount.csv')
	
        fd[fd['rating'].notnull()]['rating'] = fd[fd['rating'].notnull()]['rating'].astype('float')
        vote_averages= fd[fd['rating'].notnull()]['rating'] 
        C = vote_averages.mean()


        fd1[fd1['rating'].notnull()]['rating'] = fd1[fd1['rating'].notnull()]['rating'].astype('float')
        vote_counts = fd1[fd1['rating'].notnull()]['rating']
        m = len(vote_counts)

        md['ratings_count'] = fd1['rating']
        md['average_rating'] = fd['rating']

        qualified = md[(md['ratings_count'].notnull())][['book_id','title', 'authors', 'ratings_count', 'average_rating']]

        qualified['ratings_count'] = qualified['ratings_count'].astype('float')

        qualified['average_rating'] = qualified['average_rating'].astype('float')

        qualified.shape

        def weighted_rating(x):
            v = x['ratings_count']
            R = x['average_rating']
            return (v/(v+m) * R) + (m/(m+v) * C)

        qualified['popularity_rating'] = qualified.apply(weighted_rating, axis=1)
        pop = qualified[['book_id','popularity_rating']]
        print(qualified.shape)
        print(pop.shape)

        return pop
    ### Collaborative ##

    def collaborative(self,ratings,user_id):

        reader = Reader()
        #ratings.head()

        temp_ratings = ratings



        data = Dataset.load_from_df(temp_ratings[['user_id', 'book_id', 'rating']], reader)
#         data.split(n_folds=2)

        ## Training the data ##
        svd = SVD()
#         evaluate(svd, data, measures=['RMSE', 'MAE'])
        cross_validate(svd, data, measures=['RMSE', 'MAE'])

        trainset = data.build_full_trainset()

        algo = SVD()
        algo.fit(trainset)

        #svd.train(trainset)
        ## Testing the data ##

        from collections import defaultdict
        testset = trainset.build_anti_testset()
        predictions = algo.test(testset)

        count = 0
     
        for uid, iid, true_r, est, _ in predictions:

             if uid == user_id:
                count = count+1
                temp_ratings.loc[len(temp_ratings)+1]= [uid,iid,est]

        #print("count\n")
        #print(count)
        #print("\n--------here-------\n")	
        #print(temp_ratings)

        cb = temp_ratings[(temp_ratings['user_id'] == user_id)][['book_id', 'rating']]
        #print("\n--------here-------\n")
        #print(cb)
        
        cb = temp_ratings[(temp_ratings['user_id'] == user_id)][['book_id', 'rating']]

        return(cb)


    ##### CONTENT ######

    def content_based(self,md,ratings,user_id):       

        md['book_id'] = md['book_id'].astype('str')
        md['year'] = md['year'].astype('str')
        ratings['book_id'] = ratings['book_id'].astype('str')
        ratings['user_id'] = ratings['user_id'].astype('int')
        ratings['rating'] = ratings['rating'].astype('int')
        md['authors'] = md['authors'].str.replace(' ','')
        md['authors'] = md['authors'].str.lower()
#         md['authors'] = md['authors'].str.replace('.',' ')
#         md['authors'] = md['authors'].str.replace('\'',' ')
        md['authors'] = md['authors'].str.replace(',',' ')
        
        #print(md.head())

        md['authors'] = md['authors'].apply(lambda x: [x,x])
        md['year'] = md['year'].apply(lambda x: [x])
        md['year'] = md['year'].apply(lambda x: [x])
        #print(md['authors'])

#         md['year']=md['year'].str.split(';')
        #print(md['Genres'])
    
        
#         print(md['authors'])
#         print(md['year'])

        md['soup'] = md['authors'] + md['year']
#         print(md['soup'])

        md['soup'] = md['soup'].str.join(' ')

        #md['soup'].fillna({})
        #print(md['soup'])

        count = CountVectorizer(analyzer='word',ngram_range=(1,1),min_df=0, stop_words='english')
        count_matrix = count.fit_transform(md['soup'].values.astype('U'))
        print (count_matrix.shape)
        #print np.array(count.get_feature_names())
        #print(count_matrix.shape)


        cosine_sim = cosine_similarity(count_matrix, count_matrix)

        def build_user_profiles():
            user_profiles=np.zeros((278858 ,1000))
		#taking only the first 100000 ratings to build user_profile
            for i in range(0,45000):
#                 print(i)
                u=ratings.iloc[i]['user_id']
                b=ratings.iloc[i]['book_id']
                b2=self.md.index[self.md['book_id'] == b].tolist()[0]
		
                user_profiles[u][b2-1]=ratings.iloc[i]['rating']
                
            return user_profiles

        user_profiles=build_user_profiles()
        print('profiles done')

        def _get_similar_items_to_user_profile(person_id):
            #Computes the cosine similarity between the user profile and all item profiles

            user_ratings = np.empty((1000,1))
            cnt=0
            for i in range(0,999):
                book_sim=cosine_sim[i]
                user_sim=user_profiles[person_id]
                user_ratings[i]=(book_sim.dot(user_sim))/sum(cosine_sim[i])
            maxval = max(user_ratings)
            print(maxval)

            for i in range(0,999):
                user_ratings[i]=((user_ratings[i]*10.0)/(maxval))

                if(user_ratings[i]>3):

                    cnt+=1

            return user_ratings

        content_ratings = _get_similar_items_to_user_profile(user_id)



        num = md[['book_id']]
        num1 = pd.DataFrame(data=content_ratings[0:,0:])
        frames = [num, num1]


        content_rating = pd.concat(frames, axis =1,join_axes=[num.index])
        content_rating.columns=['book_id', 'content_rating']
        #print(content_rating.shape)
        #print(content_rating)

        return(content_rating)

    
    def final_hybrid(self,md, popularity_rating , collaborative_rating, content_rating, user_id):

        hyb = md[['book_id']]
        title = md[['book_id','title', 'year']]

        hyb = hyb.merge(title,on = 'book_id')
        hyb = hyb.merge(self.collaborative_rating,on = 'book_id')
        hyb = hyb.merge(self.popularity_rating, on='book_id')
        hyb = hyb.merge(self.content_rating, on='book_id')

        def weighted_rating(x):
            v = x['rating']
            R = x['popularity_rating']
            c = x['content_rating']
            return 0.4*v + 0.2*R + 0.4 * c


        hyb['hyb_rating'] = hyb.apply(weighted_rating, axis=1)
        hyb = hyb.sort_values('hyb_rating', ascending=False).head(999)
        hyb.columns = ['Book ID' , 'Title', 'year', 'Collaborative Rating', 'Popularity Rating' , 'Content Rating', 'Hybrid Rating']
        self.res=hyb
        print(len(hyb['Hybrid Rating']))
        print(hyb)

In [57]:
ratings = pd.read_csv('dataset/processed/FinalRatings.csv')[0:45000]
userId=276796
print('\n----------------Results for'+str(userId)+'-------------------')
h = hybrid(userId,ratings)


----------------Results for276796-------------------
       user_id     book_id  rating
0       276746  0449006522       0
1       276746  055356451X       0
2       276755  0451166892       5
3       276796  0330332775       5
4       276847  0446364193       0
...        ...         ...     ...
44995   247055  0446612545       0
44996   247055  051513290X       0
44997   247055  0553278746       0
44998   247055  0553576925       0
44999   247055  0671004573       0

[45000 rows x 3 columns]
   user_id     book_id  rating
3   276796  0330332775       5
STARTING POPULARITY
(1000, 6)
(1000, 2)
STARTING CONTENT
(1000, 870)
profiles done
[0.34045244]
STARTING COLABORATIVE
STARTING HYBRID
999
        Book ID                                              Title    year  \
491  0671002481                  The First Wives Club Movie Tie In  [1996]   
165  0451206673                                           Pen Pals  [2002]   
303  0375719180            The Miracle Life of Edgar Mint: A Novel

## RESULTS

In [58]:
h.res.head(20)

,Book ID,Title,year,Collaborative Rating,Popularity Rating,Content Rating,Hybrid Rating
491,0671002481,The First Wives Club Movie Tie In,[1996],2.561659,2.943613,10.000000,5.613386
165,0451206673,Pen Pals,[2002],4.917951,2.943586,6.400571,5.116126
303,0375719180,The Miracle Life of Edgar Mint: A Novel,[2002],5.000000,2.943586,1.326358,3.119261
187,3257233051,Veronika Deschliesst Zu Sterben / Vernika Deci...,[2002],4.853165,2.943569,1.326358,3.060523
408,0316666343,The Lovely Bones: A Novel,[2002],4.874474,2.943569,1.280114,3.050549
205,0316693006,Four Blind Mice,[2002],5.000000,2.943604,1.090083,3.024754
246,0151008116,Life of Pi,[2002],4.700600,2.943586,1.280114,2.981003
487,0805063897,Nickel and Dimed: On (Not) Getting By in America,[2002],4.532518,2.943584,1.326358,2.932267
479,0375726403,Empire Falls,[2002],4.494567,2.943611,1.326358,2.917092
300,156971620X,Harlequin Valentine,[2002],4.463940,2.943586,1.196669,2.852961
